# `PulseSimulator` prototype 1

In the first prototype phase we want to figure out the interfaces/functionality for:
- Building a `PulseSimulator` instance with a custom model.
- Simulating a pulse schedule with measurement.

## 0. Imports, and configure to use JAX

In [1]:
import numpy as np
from qiskit import pulse
from qiskit_dynamics.pulse.pulse_simulator import PulseSimulator
from qiskit_dynamics import Solver

# Configure to use JAX internally
# Enables auto-jitting of pulse simulation, so speeds things up
import jax
jax.config.update("jax_enable_x64", True)
jax.config.update('jax_platform_name', 'cpu')
from qiskit_dynamics.array import Array
Array.set_default_backend('jax')

# Solver options to pass to PulseSimulator.run
solver_options = {'method': 'jax_odeint', 'atol': 1e-10, 'rtol': 1e-10}

## 1. Instantiating `PulseSimulator` with a `Solver`

Build a `Solver` instance representing a single qubit, configured for pulse simulation.

In [2]:
Z = np.array([[-1., 0.], [0., 1.]])
X = np.array([[0., 1.], [1., 0.]])

r = 0.1

static_ham = 2 * np.pi * 5 * Z / 2
drive_op = 2 * np.pi * r * X / 2

solver = Solver(
    static_hamiltonian=static_ham,
    hamiltonian_operators=[drive_op],
    hamiltonian_channels=['d0'],
    channel_carrier_freqs={'d0': 5.},
    dt=0.1,
    rotating_frame=static_ham
)

Instantiate the `PulseSimulator` with the solver. Specifying `subsystem_dims` allows the `PulseSimulator` to know the subsystem dimensions of a composite system.

In [3]:
backend = PulseSimulator(solver=solver, subsystem_dims=[2])

Simulate a schedule. 

Notes:
- Defaults to initial state being the ground state.
- Measurements are projective, occuring right as the acquire command starts.

In [4]:
%%time

from qiskit.pulse import library

sigma = 128
num_samples = 256

schedules = []

for amp in np.linspace(0., 1., 10):
    gauss = pulse.library.Gaussian(
        num_samples, amp, sigma, name="Parametric Gauss"
    )

    with pulse.build() as schedule:
        with pulse.align_right():
            # note: carrier frequency is automatically set to channel_carrier_freq if nothing
            # specified in schedule. Is this what we want?
            # This is baked into the InstructionToSchedule object. What does it do on the backends?
            #pulse.set_frequency(5., pulse.DriveChannel(0))
            pulse.play(gauss, pulse.DriveChannel(0))
            pulse.acquire(duration=1, qubit_or_channel=0, register=pulse.RegisterSlot(0))
    
    schedules.append(schedule)

job = backend.run(schedules, shots=100, solver_options=solver_options)

job.result()

CPU times: user 1.31 s, sys: 87.4 ms, total: 1.39 s
Wall time: 1.52 s


[Result(backend_name='PulseSimulator', backend_version='0.1', qobj_id='None', job_id='ef89ca4a-3084-4673-ba36-3d8d8e5b4df3', success=True, results={'counts': {'0': 100}}, date=2022-10-14T11:49:24.564063, status=None, header=None),
 Result(backend_name='PulseSimulator', backend_version='0.1', qobj_id='None', job_id='ef89ca4a-3084-4673-ba36-3d8d8e5b4df3', success=True, results={'counts': {'0': 93, '1': 7}}, date=2022-10-14T11:49:24.564078, status=None, header=None),
 Result(backend_name='PulseSimulator', backend_version='0.1', qobj_id='None', job_id='ef89ca4a-3084-4673-ba36-3d8d8e5b4df3', success=True, results={'counts': {'0': 78, '1': 22}}, date=2022-10-14T11:49:24.564083, status=None, header=None),
 Result(backend_name='PulseSimulator', backend_version='0.1', qobj_id='None', job_id='ef89ca4a-3084-4673-ba36-3d8d8e5b4df3', success=True, results={'counts': {'0': 44, '1': 56}}, date=2022-10-14T11:49:24.564086, status=None, header=None),
 Result(backend_name='PulseSimulator', backend_versio

## 2. Validation of measurements

If a schedule doesn't have a measurement, an error is raised. 

For now:
- `PulseSimulator.run` only works for a schedule with a single measurement, and outputs counts.
- We can add arguments to `PulseSimulator.run` to allow a user to control how the simulation is done, e.g. getting counts v.s. a state simulation v.s. a propagator simulation.

In [5]:
from qiskit.pulse import library

amp = 1
sigma = 10
num_samples = 64
gauss = pulse.library.Gaussian(num_samples, amp, sigma, name="Parametric Gauss")

with pulse.build() as schedule:
    pulse.play(gauss, pulse.DriveChannel(0))

job = backend.run(
    schedule, 
    shots=100, 
    solver_options=solver_options
)

QiskitError: 'At least one measurement must be present in each schedule.'

## 3. Relabeling subsystems

When building `PulseSimulator` instances from real backends, we will select a subset of qubits. A user should be able to interact with the instance by using the original qubit labels.

In [6]:
solver = Solver(
    static_hamiltonian=static_ham,
    hamiltonian_operators=[drive_op],
    hamiltonian_channels=['d1'],
    channel_carrier_freqs={'d1': 5.},
    dt=0.1,
    rotating_frame=static_ham
)

backend_relabeled = PulseSimulator(solver=solver, subsystem_dims=[2], subsystem_labels=[1])

In [7]:
with pulse.build() as schedule:
    with pulse.align_right():
        pulse.play(gauss, pulse.DriveChannel(1))
        pulse.acquire(duration=1, qubit_or_channel=1, register=pulse.RegisterSlot(1))

job = backend_relabeled.run(
    schedule, 
    shots=100, 
    solver_options=solver_options
)

job.result()

[Result(backend_name='PulseSimulator', backend_version='0.1', qobj_id='None', job_id='3cef2a06-04bc-4555-abba-c0ef23def824', success=True, results={'counts': {'0': 83, '1': 17}}, date=2022-10-14T11:49:25.989716, status=None, header=None)]

Rerun the schedule attempting to measure qubit `0`, which doesn't exist.

In [8]:
with pulse.build() as schedule:
    with pulse.align_right():
        pulse.play(gauss, pulse.DriveChannel(1))
        pulse.acquire(duration=1, qubit_or_channel=0, register=pulse.RegisterSlot(1))

job = backend_relabeled.run(
    schedule, 
    shots=100, 
    solver_options=solver_options
)

job.result()

QiskitError: 'Attempted to measure subsystem 0, but it is not in subsystem_list.'